## <center> École Polytechnique de Montréal <br> Département Génie Informatique et Génie Logiciel <br>  INF8460 – Traitement automatique de la langue naturelle <br> </center>
## <center> TP3 - Interprétation de requêtes SPARQL par des questions en langue naturelle <br>  Automne 2023 

## Identification de l'équipe:

### Groupe de laboratoire: 

### Equipe numéro : 

### Membres: 

- membre 1 (% de contribution, nature de la contribution)
- membre 2 (% de contribution, nature de la contribution)
- membre 3 (% de contribution, nature de la contribution)

* nature de la contribution: Décrivez brièvement ce qui a été fait par chaque membre de l’équipe. Tous les membres sont censés contribuer au développement. Bien que chaque membre puisse effectuer différentes tâches, vous devez vous efforcer d’obtenir une répartition égale du travail. 

## Description

Dans ce laboratoire, vous allez construire un traducteur automatique en utilisant l'architecture du Transformeur. L'idée est d'utiliser un système de traduction automatique pour traduire des requêtes en langage SPARQL vers des questions en anglais.

SPARQL est un langage d'interrogation de bases de connaissances, similaire à SQL. Les bases de connaissances sont une source de données structurées, selon les standards, modèles et langages du Web sémantique, qui permettent un accès efficace à une grande quantité d'information dans des domaines très variés. Cependant, leur accès est limité par la complexité des requêtes qui ne permet pas au public de s'en servir directement. Il est aussi difficile pour l'usager non averti de comprendre le sens d'une requête. Nous voulons donc coder un modèle de type Transformer qui permette d'interpréter une requête SPARQL sur la base de connaissances DBpedia en lui associant une question en anglais. 

Ainsi, notre système de traduction automatique prendra en entrée une requête SPARQL et produira en sortie une phrase en anglais correspondant à la question qui est posée par la requête. Par exemple :

__Entrée__ _select distinct count ( ?uri ) where { dbr:Apocalypto dbo:language ?x . ?x dbp:region ?uri }_

__Sortie attendue__ : _In how many other dbp:region do people live, whose dbo:language are spoken in dbr:Apocalypto?_

Vous avez pu constater qu'on réutilise des éléments avec le préfixe dbr /dbo/dbp qui sont associés aux données dans DBpedia et au schéma de la base de connaissances. dbr:Apocalypto est tout simplement une URI qui décrit une ressource (ou donnée) dans DBpedia. Voici l'URI en question: https://dbpedia.org/describe/?url=http%3A%2F%2Fdbpedia.org%2Fresource%2FApocalypto&sid=35407

Dans ce TP, vous reproduirez l'architecture du Transformer à l'aide de couches Keras. Vous pouvez vous inspirer de l'implémentation de certaines méthodes du tutoriel [Tensorflow](https://www.tensorflow.org/text/tutorials/transformer)

## LIBRAIRIES PERMISES
- Jupyter notebook
- NLTK
- Numpy
- Pandas
- Sklearn
- Tensorflow
- Keras
- Transformers
- Datasets
- Pour toute autre librairie, demandez à votre chargé de laboratoire

## INFRASTRUCTURE

- Vous avez accès aux GPU du local L-4818. Dans ce cas, vous devez utiliser le dossier temp (voir le tutoriel VirtualEnv.pdf)
- Vous pouvez aussi utiliser Google Colab.

## DESCRIPTION DES DONNÉES ET MÉTRIQUES D’EVALUATION

Le corpus est un corpus de 5 000 paires de questions - requêtes sur DBPedia portant sur une grande variété de thèmes plus ou moins spécifiques. Trois ensembles de données sont fournis :

- Les 4000 paires de questions – requêtes d’entrainement dans un fichier `train.csv`.
- Les 500 paires de questions – requêtes de validation dans un fichier `validation.csv`.
- Les 500 paires de questions - requêtes de test dans un fichier `test.csv`

La métrique BLEU sera utilisée pour comparer les traductions des modèles aux requêtes de référence.

## LABORATOIRE

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_text
import pathlib
import re
from nltk.translate.bleu_score import sentence_bleu
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [2]:
import os

root = os.getcwd() + "\\" # To change if needed

### 1 Préparation des données (10 points)

Il faut tout d'abord préparer les données avant de les envoyer au système de traduction. Pour cela, deux classes seront utilisées. La classe `DataLoader` servira simplement à lire les données des fichiers d'entrainement et de validation et la classe `Preprocessor` servira à pré-traiter les données dans un format attendu

In [3]:
class DataLoader:
    """
    Classe servant à charger les données en DataFrame
    """

    def __init__(self, training_path: str, validation_path: str) -> None:
        # Initialise les attributs .train et .val en chargeant 
        # les données à partir des chemins d'accès donnés en paramètre
        
        self.train = pd.read_csv(training_path, sep=',', header=0).drop(columns=['id'])

        self.val = pd.read_csv(validation_path, sep=',', header=0).drop(columns=['id'])

    def get_train(self) -> pd.DataFrame:
        # Retourne les données d'entraînement
        return self.train
    
    def get_val(self) -> pd.DataFrame:
        # Retourne les données de validation
        return self.val
    
    def get_train_sparql(self) -> pd.Series:
        # Retourne les données d'entraînement de la source
        return self.train['sparql']
    
    def get_train_english(self) -> pd.Series:
        # Retourne les données d'entraînement de la cible
        return self.train['english']
    
    def get_val_sparql(self) -> pd.Series:
        # Retourne les données de validation de la source
        return self.val['sparql']
    
    def get_val_english(self) -> pd.Series:
        # Retourne les données de validation de la cible
        return self.val['english']

#### 1.1 Pré-traitement

La classe `Preprocessor` effectuera les transformations suivantes sur les requêtes SPARQL :
- Remplacer tous les mots clés (préfixes) de la forme `dbx:` par `dbx_` (par exemple, `dbr:` devient `dbr_` et `dbo:` devient `dbo_`). Les mots clés qui doivent être pris en compte sont les suivants : `dbr`, `dbo`, `dbp` et `rdf`
- Remplacer tous les signes de ponctuation suivants par des mots : 
  - `?` deviendra `var_`
  - `{` deviendra `brack_open`
  - `}` deviendra `brack_close`
  - `(` deviendra `parent_open`
  - `)` deviendra `parent_close`
  - `.` deviendra `sep_dot`

En ce qui concerne les questions en anglais, La classe `Preprocessor` effectuera les transformations suivantes :
- Enlever les `?` à la fin des phrases
- Remplacer tous les mots clés de la forme `dbx:` par `dbx_` (par exemple, `dbr:` devient `dbr_` et `dbo:` devient `dbo_`). Les mots clés qui doivent être pris en compte sont les suivants : `dbr`, `dbo`, `dbp` et `rdf`
- Enlèvera tous les espaces inutiles avant le début et après la fin de la question 

Cette classe s'occupe aussi d'annuler le pré-traitement une fois que le Transformer aura généré une séquence, ce qui inclut notamment d'annuler les transformations indiquées ci-dessus et d'enlever les jetons de début et de fin de phrases qui auront été ajoutés par le segmenteur un peu plus bas. 

In [4]:
class Preprocessor:
    """
    Transforme et nettoie les données pour améliorer les performances du modèle
    """

    SPARQL_TRANSLATE_OBJECTS = {
        "dbr:": "dbr_",
        "dbo:": "dbo_",
        "dbp:": "dbp_",
        "rdf:": "rdf_"
    }

    SPARQL_TRANSLATE_SYMBOLS = {
        "?": "var_",
        "{": "brack_open",
        "}": "brack_close",
        "(": "parent_open",
        ")": "parent_close",
        ".": "sep_dot"
    }

    def transform_dataframe(self, data: pd.DataFrame):
        """
        Transforme les données d'une DataFrame contenant les colonnes 'english' 
        et 'sparql'. Fait appel aux fonctions `transform_sparql` et 
        `transform_english` sur les bonnes colonnes

        Args :
            - data : Données à transformer

        Returns :
            Données transformées
        """

        # Transforme les données de la colonne 'sparql'
        data['sparql'] = data['sparql'].apply(self.transform_sparql)

        # Transforme les données de la colonne 'english'
        data['english'] = data['english'].apply(self.transform_english)

        return data

    def transform_sparql(self, sparql: str):
        """
        Transforme une requête sparql en remplacant les jetons "dbx:" par "dbx_"
        et en remplacant les signes de ponctuation par leur équivalent en mots 
        tel qu'indiqué plus haut

        Args :
            sparql : Requête sparql

        Returns :
            Requête sparql transformée avec les modifications mentionnées plus haut
        """
        
        # Remplace les SPARQL_TRANSLATE_OBJECTS dans la requête sparql
        for key, value in self.SPARQL_TRANSLATE_OBJECTS.items():
            sparql = sparql.replace(key, value)
        
        # Remplace les SPARQL_TRANSLATE_SYMBOLS dans la requête sparql
        for key, value in self.SPARQL_TRANSLATE_SYMBOLS.items():
            sparql = sparql.replace(key, value)

        return sparql

    def transform_english(self, english: str):
        """
        Transforme une requête sparql en remplacant les jetons "dbx:" 
        par "dbx_" et en enlevant les points d'interrogation ainsi que 
        les espaces non-nécessaires au début et à la fin de la phrase

        Args :
            - english : Phrase en anglais sur laquelle appliquer 
            les transformations
        
        Returns :
            Phrase transformée avec les modifications mentionnées plus haut
        
        """
        
        # Remplace les SQARQL_TRANSLATE_OBJECTS dans la question en anglais
        for key, value in self.SPARQL_TRANSLATE_OBJECTS.items():
            english = english.replace(key, value)

        # Enlève les points d'interrogation
        english = english.replace('?', '')

        # Enlève les espaces au début et à la fin de la phrase
        english = english.strip()

        return english

    def transform_back_english(self, english):
        """
        Effectue les transformations inverses de la phrase en anglais 
        (remplace les dbx_ en dbx:).
        Attention, cette fonction doit aussi enlever les jetons de début 
        et de fin d'une phrase qui sont ajoutés lors de 
        la segmentation (tokenization)

        Args :
            - english : Phrase générée par un modèle contenant les jetons 
            de début et de fin
        
        Returns :
            - Phrase en anglais dont les transformations ont été annulées
        """
        english = bytes(tf.squeeze(english).numpy()).decode()
        
        # Rétablie les SPARQL_TRANSLATE_OBJECTS dans la question en anglais
        for key, value in self.SPARQL_TRANSLATE_OBJECTS.items():
            english = english.replace(value, key)

        return english



Vous pouvez vérifier votre implémentation de la classe `Preprocessor` à l'aide du test suivant

In [5]:
def test_preprocessor():

    test_queries = [
        'select distinct count ( ?uri ) where { ?uri dbo:director dbr:Stanley_Kubrick . }',
        'select distinct ?uri where { ?uri dbo:founder dbr:John_Forbes_(British_Army_officer) . ?uri rdf:type dbo:City }'
    ]

    test_english = [
        'how many movies are there whose dbo:director is dbr:Stanley_Kubrick ?',
        'what dbo:City\'s dbo:founder is dbr:John_Forbes_(British_Army_officer) ?'
    ]
    
    preprocessor = Preprocessor()
    print('Transformed sparql : ')
    for query in test_queries:
        print(preprocessor.transform_sparql(query))

    print()
    print('Transformed english : ')
    for english in test_english:
        print(preprocessor.transform_english(english))


test_preprocessor()

Transformed sparql : 
select distinct count parent_open var_uri parent_close where brack_open var_uri dbo_director dbr_Stanley_Kubrick sep_dot brack_close
select distinct var_uri where brack_open var_uri dbo_founder dbr_John_Forbes_parent_openBritish_Army_officerparent_close sep_dot var_uri rdf_type dbo_City brack_close

Transformed english : 
how many movies are there whose dbo_director is dbr_Stanley_Kubrick
what dbo_City's dbo_founder is dbr_John_Forbes_(British_Army_officer)


Sortie attendue :

```
Transformed sparql : 
select distinct count parent_open var_uri parent_close where brack_open var_uri dbo_director dbr_Stanley_Kubrick sep_dot brack_close

select distinct var_uri where brack_open var_uri dbo_founder dbr_John_Forbes_parent_openBritish_Army_officerparent_close sep_dot var_uri rdf_type dbo_City brack_close

Transformed english : 
how many movies are there whose dbo_director is dbr_Stanley_Kubrick

what dbo_City's dbo_founder is dbr_John_Forbes_(British_Army_officer)
```

Vous pouvez maintenant instancier une objet de la classe `Data Loader` pour charger les données d'entrainement et de validation à partir des fichiers `train.csv` et `validation.csv`

In [6]:
# Instancier une objet de la classe DataLoader pour charger les données
data_loader = DataLoader(
    training_path=root + 'train.csv',
    validation_path=root +'validation.csv'
)

Appliquez le pré-traitement des données sur les données chargées précédemment

In [7]:
# Appliquer le pre-processeur sur les données d'entrainement et de validation

pre_processor = Preprocessor()
processed_train = pre_processor.transform_dataframe(data_loader.train)
processed_val = pre_processor.transform_dataframe(data_loader.val)

### 2. Segmentation (tokenization) (15 points)

Une fois les données importées et modifiées, il faut adapter les phrases dans un format que le modèle peut comprendre.

Tout d'abord, il faudra segmenter les phrases en jetons. Pour cela, un dictionnaire de mots (vocabulaire) sera nécessaire. 

#### 2.0 LanguageTokenizer (10 points)

La classe `LanguageTokenizer` s'occupera de créer ce vocabulaire et de transformer les phrases d'un langage spécifique en jetons. Dans notre cas, il y aura 2 instances de cette classe : une pour l'anglais et l'autre pour sparql. Cette classe possède plusieurs fonctions qui nous seront très utiles notamment `create_vocab` pour créer le vocabulaire du modèle, `tokenize` pour transformer les phrases en jetons et `detokenize` pour transformer les jetons en phrases. 

Nous allons avoir recours au segmenteur de Bert pour trouver les jetons et le vocabulaire. Les paramètres du segmenteur vous sont donnés. Ce segmenteur divise chaque mot en parties de mots. Par exemple "characteristically" sera segmenté en 'characteristic' et '##ally'. 

Ensuite, pour chacune des phrases, après les avoir transformées en jetons, il faudra ajouter les jetons de début (`[START]`) et de fin de phrase (`[END]`). Cette opération sera effectuée dans la fonction `add_start_end`.

In [8]:
class LanguageTokenizer(tf.Module):
    """
    Classe représentant un tokenizer pour un langage spécifique. 
    Dans notre cas, il y en aura un pour sparql et un pour l'anglais
    """

    reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]
    START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
    END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

    tokenizer_params = dict(lower_case=True)

    vocab_args = dict(
        vocab_size = 8000,
        reserved_tokens=reserved_tokens,
        bert_tokenizer_params=tokenizer_params,
        learn_params=None,
    )

    def __init__(self, reserved_tokens, vocab_path):
        """
        Initialise le BertTokenizer en utilisant le paramètre `vocab_path` 
        et en mettant le tokenizer en mode "lower case".

        Args :
            - reserved_tokens : Jetons réservés du BertTokenizer
            - vocab_path : Chemin vers le fichier contenant le vocabulaire du tokenizer
        """
        
        super().__init__(name="LanguageTokenizer")

        with open(vocab_path) as f:
            f = open(vocab_path, 'r')

            init = tf.lookup.TextFileInitializer(
                    f.name,
                    key_dtype=tf.string, key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                    value_dtype=tf.int64, value_index=tf.lookup.TextFileIndex.LINE_NUMBER)

            lookup_table = tf.lookup.StaticVocabularyTable(
                init,
                num_oov_buckets=1
            )

            f.close()
        
        # Initialize the BertTokenizer
        self.tokenizer = tensorflow_text.BertTokenizer(lookup_table, **self.tokenizer_params)
        
        # Initialise les jetons réservés
        self.reserved_tokens = reserved_tokens

    def create_vocab(language_sentences: pd.DataFrame, path: str):
        """
        Crée un vocabulaire à partir des phrases en entrée 
        (language_sentences). Pour cela vous devrez utiliser 
        la fonction bert_vocab_from_dataset(). Attention, il 
        ne faut pas oublier de passer en paramètres `vocab_args` 
        à la fonction qui s'occupe de créer le vocabulaire.

        Une fois le vocabulaire créé, il faudra le sauvegarder dans un fichier spécifié
        par l'attribut `path`.

        Args :
            - language_sentences : DataFrame contenant les phrases du langage
            - path : Chemin où sera sauvegardé le vocabulaire
        """
        
        # On convertie les phrases en un dataset (Tensorflow)
        vocab_tf_dataset = tf.data.Dataset.from_tensor_slices((language_sentences))

        # Création du vocabulaire
        vocab = bert_vocab.bert_vocab_from_dataset(vocab_tf_dataset, **LanguageTokenizer.vocab_args)
        
        # Save the vocabulary to the specified file
        with open(path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(vocab))
            f.close()

    @tf.function
    def tokenize(self, inputs):
        """
        Transforme des phrases en index de jetons et qui ajoute les 
        jetons de début et de fin.
        
        Args :
            - inputs : Phrases d'entrée

        Returns :
            Jetons correspondant à la phrase avec les jetons de début et de fin
        """

        # Tokenize the inputs into a ragged tensor
        tokens = self.tokenizer.tokenize(inputs).merge_dims(1,2) # TODO: CHECK IF THERE'S A WAY TO AVOID THE MERGE
        
        # Add the start and end token to the tokenized inputs
        tokens = LanguageTokenizer.add_start_end(tokens)

        return tokens

    @tf.function
    def detokenize(self, tokenized):
        """
        Transforme une liste d'index en jetons. Applique ensuite 
        la méthode `cleanup_text` du pour nettoyer 
        les données.

        Args :
            - tokenized : Liste de jetons

        Returns :
            Phrase correspondant aux jetons
        """

        # Detokenize: turn the token IDs back into text returned as a ragged tensor
        detokenized = self.tokenizer.detokenize(tokenized)

        # Cleaning the detokenized text tensor back into a regular string
        detokenized = LanguageTokenizer.cleanup_text(self.reserved_tokens, detokenized)

        return detokenized

    def add_start_end(tokenized_sentences):
        """
        Fonction qui ajoute la représentation des tokens [START] et [END] à la phrase en entrée
        
        Args :
            - tokenized_sentences: Tenseur contenant les indices des jetons des phrases

        Returns :
            Tenseur initial avec les indices des jetons [START] et [END] au début et à la fin
        """

        # Create the start and end tokens tensors. Dimensions should match the number of sentences
        start = tf.fill([tokenized_sentences.bounding_shape()[0], 1], LanguageTokenizer.START)
        end = tf.fill([tokenized_sentences.bounding_shape()[0], 1], LanguageTokenizer.END)

        # Make the data type compatible with the tokenized_sentences tensor
        start = tf.cast(start, tf.int64)
        end = tf.cast(end, tf.int64)

        # Concatenate the start tokens, sentences and end tokens
        tokenized_sentences = tf.concat([start, tokenized_sentences, end], axis=1)

        return tokenized_sentences


    def cleanup_text(reserved_tokens, token_txt):
        """
        Fonction qui nettoie un texte généré par la fonction detokenize() du BertTokenizer.
        Args :
            - reserved_tokens : Jetons réservés du BertTokenizer
            - token_text : Chaine généré par la fonction detokenize()

        Returns :
            Texte nettoyé
        """
        
        # Determine which tokens are reserved based on the reserved_tokens
        bad_tokens = [re.escape(token) for token in reserved_tokens if token != "[UNK]"] # Tokens to remove are all non-[UNK] tokens

        # Create a regular expression pattern from the bad_tokens
        bad_token_regex = tf.strings.join(bad_tokens, "|")

        # Determine which tokens match the bad_token_regex
        bad_cells = tf.strings.regex_full_match(token_txt, bad_token_regex)

        # Replace the bad tokens with an empty string
        cleaned_cells = tf.ragged.boolean_mask(token_txt, ~bad_cells)

        # Joining the good cells back into text
        cleaned_cells = tf.strings.reduce_join(cleaned_cells, separator=' ', axis=-1)

        return cleaned_cells

In [9]:
def test_add_start_end():

    tokenized_sentence = tf.ragged.constant([[320, 24, 500, 23, 21], [43, 45, 102, 30]], dtype=tf.int64)
    tf.print(LanguageTokenizer.add_start_end(tokenized_sentence))

test_add_start_end()

[[2, 320, 24, ..., 23, 21, 3], [2, 43, 45, 102, 30, 3]]


Sortie attendue : 
```
[[2, 320, 24, ..., 23, 21, 3], [2, 43, 45, 102, 30, 3]]
```

In [10]:
def test_tokenizer():

    sentence = ['how many U.S Presidents were born in New York ?']
    vocab_path = root + 'test_language_vocab.txt'
    LanguageTokenizer.create_vocab(sentence, vocab_path)

    with open(vocab_path) as f:
        vocab = f.read()

    print('Vocabulary : ', vocab.replace('\n', ' '))
    test_tokenizer_obj = LanguageTokenizer(LanguageTokenizer.reserved_tokens, vocab_path)
    tokenized_sentence = test_tokenizer_obj.tokenize(sentence)
    tf.print(f'Tokenized sentence : {tokenized_sentence}')

    detokenized_sentence = test_tokenizer_obj.detokenize(tokenized_sentence)
    tf.print(f'Detokenized sentence : {bytes(tf.squeeze(detokenized_sentence).numpy()).decode()}') # TODO: Would fail for batched inputs

test_tokenizer()

Vocabulary :  [PAD] [UNK] [START] [END] . ? a b d e h i k m n o p r s t u w y ##. ##? ##a ##b ##d ##e ##h ##i ##k ##m ##n ##o ##p ##r ##s ##t ##u ##w ##y
Tokenized sentence : <tf.RaggedTensor [[2, 10, 34, 40, 13, 25, 33, 41, 20, 4, 18, 16, 36, 28, 37, 30, 27, 28,
  33, 38, 37, 21, 28, 36, 28, 7, 34, 36, 33, 11, 33, 14, 28, 40, 22, 34,
  36, 31, 5, 3]]>
Detokenized sentence : how many u . s presidents were born in new york ?


Sortie attendue : 
```
Vocabulary :  [PAD] [UNK] [START] [END] . ? a b d e h i k m n o p r s t u w y ##. ##? ##a ##b ##d ##e ##h ##i ##k ##m ##n ##o ##p ##r ##s ##t ##u ##w ##y 
Tokenized sentence : <tf.RaggedTensor [[2, 10, 34, 40, 13, 25, 33, 41, 20, 4, 18, 16, 36, 28, 37, 30, 27, 28,
  33, 38, 37, 21, 28, 36, 28, 7, 34, 36, 33, 11, 33, 14, 28, 40, 22, 34,
  36, 31, 5, 3]]>
Detokenized sentence : how many u . s presidents were born in new york ?
```

#### 2.1 Vocabulaire  (5 points) 

Vous pouvez maintenant créer le vocabulaire de chaque langage à l'aide de la fonction `create_vocab`. Vous pouvez stocker le vocabulaire anglais dans un fichier appelé `language_vocab_english.txt` et le vocabulaire sparql dans un fichier appelé `language_vocab_sparql.txt`

In [11]:
sparql_vocab_path = root + 'language_vocab_sparql.txt'
english_vocab_path = root + 'language_vocab_english.txt'

# Create the vocabulary for the sparql and english sentences
LanguageTokenizer.create_vocab(processed_train['sparql'], sparql_vocab_path)
LanguageTokenizer.create_vocab(processed_train['english'], english_vocab_path)


Afin de n'utiliser qu'une seule classe, nous allons créer une classe qui regroupe les deux tokenizers en une seule classe appelée `GroupedTokenizers`. Complétez le constructeur qui initialise l'attribut `english` correspondant au tokenizer anglais et l'attribut `sparql` correspondant au tokenizer sparql.

In [12]:
class GroupedTokenizers(tf.Module):
    """
    Cette classe regroupe les deux segmenteurs (tokenizers) qui seront 
    utilisés (une pour chacun des langages)
    """

    def __init__(self, reserved_tokens, vocab_english_path: str, vocab_sparql_path: str):
        """
        Initialise les deux tokenizers (english and sparql)
        Args :
            - reserved_tokens : Jetons réservés du BertTokenizer
            - vocab_english_path : Chemin vers le fichier contenant 
            le vocabulaire anglais du segmenteur (tokenizer)
            - vocab_sparql_path : Chemin vers le fichier contenant 
            le vocabulaire sparql du segmenteur (tokenizer)
        """
        self.english = LanguageTokenizer(reserved_tokens, vocab_english_path)
        self.sparql = LanguageTokenizer(reserved_tokens, vocab_sparql_path)
        pass

Le test suivant permet de vérifier que votre pré-traitement et votre tokenizer fonctionnent correctement

In [13]:
tokenizers = GroupedTokenizers(
    LanguageTokenizer.reserved_tokens,
    root + 'language_vocab_english.txt',
    root + 'language_vocab_sparql.txt'
)

def test_tokenizer_preprocessor(tokenizers: GroupedTokenizers):
    """
    Verifie que les fonctions du tokenizer et du preprocessor sont belles 
    et bien codées. Si elles le sont, les phrases initiales anglaises et 
    sparql devraient être identiques à celles en entrée

    """
    english = 'how many movies are there whose dbo:director is dbr:Stanley_Kubrick ?'
    sparql = 'select distinct count ( ?uri ) where { ?uri dbo:director dbr:Stanley_Kubrick . }'
    print('English : \n', english, '\n')

    # processed_train = pre_processor.transform_dataframe
    pre_processor = Preprocessor()
    processed_english = pre_processor.transform_english(english)
    processed_sparql = pre_processor.transform_sparql(sparql)

    print('Processed english : \n', processed_english, '\n')
    tokenized_english = tokenizers.english.tokenize(processed_english)
    print('Tokenized english : \n', tokenized_english, '\n')
    detokenized_english = pd.Series(tokenizers.english.detokenize(tokenized_english).numpy())
    print('Detokenized english : \n', detokenized_english.apply(pre_processor.transform_back_english), '\n')
    print()
    print('------------------------------------------------')
    print()

    print('Sparql : \n', sparql, '\n')

    print('Processed sparql : \n', processed_sparql, '\n')
    tokenized_sparql = tokenizers.sparql.tokenize(processed_sparql)
    print('Tokenized sparql : \n', tokenized_sparql, '\n')

test_tokenizer_preprocessor(tokenizers)

English : 
 how many movies are there whose dbo:director is dbr:Stanley_Kubrick ? 

Processed english : 
 how many movies are there whose dbo_director is dbr_Stanley_Kubrick 

Tokenized english : 
 <tf.RaggedTensor [[2, 74, 75, 495, 67, 73, 65, 61, 25, 228, 59, 60, 25, 896, 95, 261, 25,
  36, 116, 329, 757, 114, 3]]> 

Detokenized english : 
 0    how many movies are there whose dbo _ director...
dtype: object 


------------------------------------------------

Sparql : 
 select distinct count ( ?uri ) where { ?uri dbo:director dbr:Stanley_Kubrick . } 

Processed sparql : 
 select distinct count parent_open var_uri parent_close where brack_open var_uri dbo_director dbr_Stanley_Kubrick sep_dot brack_close 

Tokenized sparql : 
 <tf.RaggedTensor [[2, 66, 65, 71, 68, 22, 63, 55, 22, 56, 68, 22, 62, 64, 57, 22, 63, 55,
  22, 56, 61, 22, 208, 59, 22, 41, 182, 80, 233, 22, 33, 879, 703, 111,
  60, 22, 58, 57, 22, 62, 3]]> 



### 3. Création de lots (Batching) (5 points)

Étant donnée la grande quantité de données impliquant l'entrainement d'un modèle, il est important d'envoyer les données de la manière la plus efficace possible. Pour cela, les données sont regroupées en petits groupes appelés "batchs" ou lots. Cela permet notamment de traiter plusieurs éléments en parallèle et réduit considérablement le temps d'entrainement.

Pour cela, la classe `Batcher` sera utilisée. Cette classe s'occupe de regrouper les données en petits lots et de les préparer pour les envoyer au modèle. Cette classe possède plusieurs fonctions :
- `make_batches`: Elle reçoit en paramètre une instance de la classe `tf.Dataset`. Elle divise ensuite le dataset en petits lots et les envoie à la fonction `prepare_batch` 
- `prepare_batch`: Reçoit un lot/"batch" et le prépare en effectuant les transformations suivantes :
  - Segmente les phrases en entrée en utilisant les bons tokenizers passés en paramètres dans le constructeur
  - S'assure que la taille des phrases ne dépasse pas `max_tokens`


<img src="Batcher.png" alt="Batcher" width="100%" height="700"/>

In [139]:
class Batcher():
    """
    Cette classe s'occupe de regrouper les données en petits groupes (batches) et 
    de préparer les données pour les envoyer au modèle.
    """

    def __init__(self, tokenizers: GroupedTokenizers, train, max_tokens, batch_size, buffer_size):
        """
        Initialise les paramètres en entrée 

        Args :
            - tokenizers : tokenizers pour transformer les entrées en jeton
            - train : Valeur booléenne pour savoir si les batches seront utilisées 
            pour de l'entrainement ou pas
            - max_tokens : Nombre de jetons maximums pour une entrée
            - batch_size : Taille des groupes (batches)
            - buffer_size : Taille du buffer servant à mélanger les données dans le 
            cas de l'entrainement
        """

        self.tokenizers = tokenizers
        self.train = train
        self.max_tokens = max_tokens
        self.buffer_size = buffer_size
        self.batch_size = batch_size

    def prepare_batch(self, input_language, output_language=None):
        """
        Prépare les batches pour les envoyer au modèle. Cette fonction est 
        appelée pour chaque élément d'un Tensorflow Dataset.

        Effectue les transformations suivantes :
            - Tokenize les phrases en entrées en utilisant les bons tokenizers passés 
            en paramètre dans le constructeur
            - S'assure que la taille des phrases ne dépasse pas `max_tokens` (max_tokens 
            est inclus)

        Args :
            - input_language : Entrée dans le langage d'entrée (sparql dans notre cas) 
            de la taille (self.batch_size, x)
            - output_language : Sortie dans le langage de sortie (english dans notre cas) 
            de la taille (self.batch_size, x). None dans le cas de batches de test

        Returns :
            - Si self.train == True :
                Retourne un tuple de la forme ((input_langage, output_language_inputs), output_language_labels)
                qui seront les entrées respectives de l'encodeur et du décodeur et la 
                sortie du décodeur.

                Voici ce que chaque valeur de retour représente
                - input_language : tenseur contenant les jetons du paramètre `input_language` 
                limité à `max_tokens`
                - output_language_inputs : tenseur contenant les jetons du paramètre 
                `output_language` limité à `max_tokens`+1 (pour permettre de prédire le prochain 
                jeton)
                - output_language_labels : tenseur contenant les jetons du paramètre 
                `output_language` contenant le prochain charactère
                
            - Si self.train == False :
                Retourne un tuple de la forme (input_language, output_language) qui 
                représentent l'entrée de l'encodeur et un
                tenseur de sortie initialisé avec le jeton d'entrée de la taille 
                (self.batch_size,). Les valeurs de retour sont expliquées plus haut
        """
        # TODO: wait for moodle answer before fixing bug
        input_language = self.tokenizers.sparql.tokenize(input_language)
        input_language = input_language.to_tensor()
        input_language = input_language[:, :self.max_tokens]

        if (self.train == True):
            output_language_inputs = self.tokenizers.english.tokenize(output_language)
            output_language_inputs = output_language_inputs.to_tensor()
            output_language_labels = output_language_inputs[:, :self.max_tokens+1]
            output_language_inputs = output_language_inputs[:, :self.max_tokens]
            return ((input_language, output_language_inputs), output_language_labels)
        else:
            output_language = tf.fill([input_language.shape[0], 1], LanguageTokenizer.START)
            return (input_language, output_language)
    
    def make_batches(self, ds):
        """
        Args :
            - ds : Dataset contenant les examples de la forme 
            ((sparql, english_in), english_label)
            si self.train == True et de la forme (sparql, english) 
            si self.train == False

        Returns :
            Le dataset initial (mélangé si self.train == True) contenant des 
            éléments de la taille de self.batch_size dont la fonction self.prepare_batch 
            a été appelée sur chacun des éléments et dont les éléments sont 
            pré-récupéré (prefetched). Si self.train == False, c'est le même principe,
            mais les données ne sont pas mélangées
        """
        # TODO: wait for moodle answer before fixing bug
        if (self.train == True):
            ds = ds.shuffle(self.buffer_size)
        ds = ds.map(self.prepare_batch)
        ds = ds.batch(self.batch_size)
        return ds

Vous pouvez maintenant tester le batcher à l'aide de la fonction suivante (vérifiez bien que la sortie du décodeur contient un jeton de plus que la phrase qui entre dans le décodeur et que ce qui rentre dans l'encodeur est bel et bien du sparql).

In [55]:
def test_batcher(tokenizers):


    english = pd.Series([
        'how many movies are there whose dbo_director is dbr_Stanley_Kubrick', 
        'what is the dbo_River whose dbo_riverMouth is dbr_Dead_Sea', 
    ])

    sparql = pd.Series([
        'select distinct count parent_open var_uri parent_close where brack_open var_uri dbo_director dbr_Stanley_Kubrick sep_dot brack_close', 
        'select distinct var_uri where brack_open var_uri dbo_riverMouth dbr_Dead_Sea sep_dot var_uri rdf_type dbo_River brack_close', 
    ])
    
    batcher = Batcher(tokenizers, True, 8, 64, 20000)

    val_english = tf.data.Dataset.from_tensor_slices(english)
    val_sparql = tf.data.Dataset.from_tensor_slices(sparql)
    val_examples = tf.data.Dataset.zip((val_sparql, val_english))

    batches = batcher.make_batches(val_examples)
    for x in batches:
        print('yoo')
        print(x[0][0].shape)
        print(x[0][1].shape)
        print(x[1].shape)
        tf.print('Detokenized inputs encoder : ', tokenizers.sparql.detokenize(x[0][0]))
        tf.print('Detokenized inputs decoder  : ', tokenizers.english.detokenize(x[0][1]))
        tf.print('Detokenized outputs decoder : ', tokenizers.english.detokenize(x[1]))

        concat = tf.concat([x[0][0], x[0][1], x[1]], axis=1)
        print('Concatened values : ', concat)

test_batcher(tokenizers)

yoo
(2, 1, 8)
(2, 1, 8)
(2, 1, 9)
Detokenized inputs encoder :  [['select distinct count parent _ open var'], ['select distinct var _ uri where brack']]
Detokenized inputs decoder  :  [['how many movies are there whose dbo'], ['what is the dbo _ river whose']]
Detokenized outputs decoder :  [['how many movies are there whose dbo _'], ['what is the dbo _ river whose dbo']]


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Dimension 2 in both shapes must be equal: shape[0] = [2,1,8] vs. shape[2] = [2,1,9] [Op:ConcatV2] name: concat

### 4. Transformer (30 points)

<img style="float: right;" src="Transformer.png" alt="Transformer" width="500" height="700"/>


Maintenant que les données sont prêtes à être envoyées au modèle, il ne manque qu'à créer son architecture. Pour cela, la librairie Keras sera utilisée. Keras est une librairie qui est construite au dessus de Tensorflow pour faciliter le développement de modèles dans un style orienté objet. Depuis Tensorflow 2.0, elle est maintenant directement intégrée à Tensorflow. Pour plus de détails, la documentation est présente sur ce [site](https://keras.io/api/)

L'architecture qui sera suivie dans ce TP est présentée dans l'image à droite. La liste des couches qui seront implémentées sont les suivantes : 
- `Positional Embedding` : Permet la génération des plongements de position
- `Global-Self Attention` : S'occupe du mécanisme d'attention de l'encodeur
- `Feed Forward` : Permet de connecter des entrées et des sorties avec un réseau de neurones
- `Decoder Attention` : S'occupe du premier mécanisme d'attention du décodeur
- `Cross Attention` : S'occupe du deuxième mécanisme d'attention du décodeur (relie l'encodeur au décodeur)

Les couches d'addition et de normalisation seront incluses dans les couches précédentes. Par exemple, la couche `Add & Norm` qui suit la couche `Global-Self Attention` dans le graphique sera inclus dans la couche `Global-Self Attention`.

Ensuite, des couches seront égalements utilisées pour regrouper ces couches pour simplifier le pipeline du Transformer. Voici la liste des couches qui seront ajoutées à celles sur le graphique :
- `Encoder Layer` : Représente un seul encodeur contenant les couches `Global-Self Attention` et `Feed Forward`
- `Decoder Layer` : Représente un seul décodeur contenant les couches `Decoder Attention`, `Cross Attention` et `Feed Forward` 
- `Encoder` : Représente plusieurs encodeurs en parallèle 
- `Decoder` : Représente plusieurs décodeurs en parallèle
- `Transformer` : Représente le Transformer au complet et regroupe tous les encodeurs, décodeurs et les couches de plongements

Chaque couche sera créée manuellement et implémentée en tant que couche Keras. Si vous n'êtes pas familier avec Keras, voici quelques tutoriels qui pourraient vous aider :
- https://keras.io/api/models/model/
- https://www.tensorflow.org/text/tutorials/transformer
- https://machinelearningmastery.com/implementing-the-transformer-encoder-from-scratch-in-tensorflow-and-keras/


Les classes sont déjà créées pour vous et vous n'aurez principalement qu'à implémenter la fonction `call()` de chacune de ces classes. La fonction `call()` s'occupe, pour une couche donnée, de transformer une entrée en sortie.

#### 4.1 Positional Embedding  

Pour permettre au modèle de prendre en compte l'ordre des jetons qui lui sont passés, il est important de passer de l'information au modèle à propos de la position des jetons dans une phrase. C'est la couche de `PositionalEmbedding` qui s'occupe de cela. À l'aide de la formule suivante, des plongements de position sont générés, ce qui permet d'incorporer la position d'un jeton dans son plongement : 
$$PE_{(pos, 2i)} = sin \Big( \frac{pos}{10000^{2i/d_{model}}} \Big)$$
$$PE_{(pos, 2i+1)} = cos \Big( \frac{pos}{10000^{2i/d_{model}}} \Big)$$

où $d_{model}$ est la dimension des plongements de sortie et $i$ est simplement l'indice d'une valeur dans le vecteur de plongement.

La fonction `generate_positional_embedding` qui génère les plongements de position vous est fournie. Celle-ci prend en entrée :
- `length` : Nombre de jetons maximal dont on doit générer le plongement de position
- `depth` : Dimension des plongements du modèle.

La fonction `call` de cette couche est appelée avec le paramètre suivant (les tailles des tenseurs sont indiquées entre parenthèses) :
- `x` (de taille [batch_size, input_size] où le batch_size est le nombre d'éléments qui sont envoyés à la fois pour une itération de l'entraînement et input_size est la taille maximale des phrases en entrée) : Entrées de la couche. Cela correspond notamment au tenseur contenant les indices de chaque jeton correspondant à la phrase
  

Elle retourne le plongement de l'entrée dans l'espace latent incluant les positions des jetons (batch_size, input_size, dim_model). 

La fonction `call` doit effectuer les opérations suivantes :
1. Appeler la couche `embedding_layer` qui génère des plongements par rapport aux entrées
2. Multiplier chaque valeur par la racine de `dim_model` (Cette multiplication sert à agrandir les plongements pour qu'ils soient d'un ordre de grandeur comparable aux plongements de position qui sont ajoutés par la suite. Pour plus de détails, consultez l'article original ayant mené à la création du Transformer intitulé "Attention Is All You Need").
3. Ajouter ensuite les plongements de positions aux plongements générés par la couche `embedding_layer` (après qu'ils aient été multipliés par la racine de `dim_model`)

In [58]:
class PositionalEmbedding(tf.keras.layers.Layer):
    """
    Classe représentant l'étape qui incorpore dans l'espace latent les positions des jetons
    """
    def __init__(self, input_size, dim_model):
        """
        Initialise une couche de plongements et les plongements de position

        Args :
            - input_size : Taille d'entrée de la couche (taille du vocabulaire)
            - dim_model : Taille des plongements du modèle (taille du plongement de sortie de la couche)
        """
        super().__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_size, dim_model, mask_zero=True)
        self.position_embeddings = self.generate_positions_embedding(length=2048, depth=dim_model)
        self.dim_model = dim_model

    def compute_mask(self, *args, **kwargs):
        return self.embedding_layer.compute_mask(*args, **kwargs)

    def generate_positions_embedding(self, length, depth):
        depth = depth/2

        positions = np.arange(length)[:, np.newaxis]
        depths = np.arange(depth)[np.newaxis, :]/depth

        angle_rates = 1 / (10000**depths)
        angle_rads = positions * angle_rates

        pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)

        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, x):
        """
        Exécute la couche de plongements sur l'entrée en la normalisant sur la racine de la dimension de sortie
        """
        x = self.embedding_layer(x) * tf.math.sqrt(tf.cast(self.dim_model, tf.float32))
        x = x + self.position_embeddings[tf.newaxis, :tf.shape(x)[1], :]
        return x

#### 4.2 Attention  (15 points)

Les couches d'attention reposent toutes sur la même base qui contient une tête d'attention multiple, une couche de normalisation et une couche d'addition. La seule différence entre les différentes couches d'attention sont les entrées `Q` (query), `K` (key), et `V` (value) qui seront envoyées à la formule :

$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

Pour cela, la classe `DefaultAttention`, une classe dont toutes les autres couches d'attention hériteront, a été créée pour éviter de répéter 3 fois le même constructeur. Vous devez compléter les fonctions `call()` de chacune des sous-classes, soit `CrossAttention`, `GlobalSelfAttention` et `DecoderAttention`. Pour évaluer les valeurs de `K`, `V` et `Q` de chaque couche d'attention, référez-vous au graphique de l'architecture.


In [59]:
class DefaultAttention(tf.keras.layers.Layer):
    """
    Couche d'attention de base contenant des têtes d'attention suivies d'une couche de normalisation et d'addition
    """
    def __init__(self, **kwargs):
        super().__init__()
        self.multiHeadAttention = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layerNormalization = tf.keras.layers.LayerNormalization()
        self.addLayer = tf.keras.layers.Add()



##### 4.2.1 CrossAttention (5 points)
Dans le cas de la couche `CrossAttention`, la fonction `call` prend en paramètres les entrées suivantes :
- `input` : Les entrées de la couche, correspondant à la sortie de la couche `DecoderAttention`
- `context` : La sortie de l'encodeur 
- `training` : Valeur booléenne indiquant si le modèle est en entraînement ou pas.

Cette fonction doit exécuter les opérations suivantes :
1. Appliquer la couche de têtes d'attention multiples avec les bonnes valeurs de `K`, `V` et `Q` (Ne pas oublier de passer l'argument `training` à la couche).
2. Ajouter la sortie de la couche de tête d'attention aux entrées à l'aide de la couche `Add`
3. Normaliser le tout à l'aide de la couche de normalisation

In [102]:
class CrossAttention(DefaultAttention):
    """
    Couche qui connecte l'encodeur au décodeur.
    """

    def __init__(self, **kwargs):
        """
        Initialise une couche de têtes d'attention suivie d'une couche de normalisation 
        puis d'addition
        """
        super().__init__(**kwargs)

    def call(self, input, context, training):
        """
        Exécute la couche d'attention. Ajoute les sorties d'attention à l'entrée et 
        normalise le tout
        """
        # TODO 
        Q = input
        K = context
        V = context
        attention_output = self.multiHeadAttention(Q, K, V, training=training)
        Q = self.addLayer([Q, attention_output])
        Q = self.layerNormalization(Q)
        return Q



##### 4.2.2 GlobalSelfAttention  (5 points)
Dans le cas de la couche `GlobalSelfAttention`, la fonction `call` prend en paramètres les entrée suivantes :
- `input` : Les entrées de la couche, correspondant à la sortie de la couche `DecoderAttention`
- `training` : Valeur booléenne indiquant si le modèle est en entraînement ou pas.

Cette fonction doit exécuter les opérations suivantes :
1. Appliquer la couche de têtes d'attention multiples avec les bonnes valeurs de `K`, `V` et `Q` (Ne pas oublier de passer l'argument `training` à la couche).
2. Ajouter la sortie de la couche de tête d'attention aux entrées à l'aide de la couche `Add`
3. Normaliser le tout à l'aide de la couche de normalisation

In [101]:
class GlobalSelfAttention(DefaultAttention):
    """
    Couch d'auto-attention permettant au modèle de regarder les autres mots de 
    la phrase d'entrée lorsqu'il encode un mot spécifique
    """

    def __init__(self, **kwargs):
        """
        Initialise une couche de têtes d'attention suivie d'une couche de 
        normalisation puis d'addition
        """
        super().__init__(**kwargs)

    def call(self, input, training):
        """
        Exécute la couche d'attention. Ajoute les sorties d'attention à l'entrée 
        et normalise le tout
        """
        # TODO
        Q, K, V = input, input, input
        attention_output = self.multiHeadAttention(Q, K, V, training=training)
        Q = self.addLayer([Q, attention_output])
        Q = self.layerNormalization(Q)
        return Q


##### 4.2.3 DecoderAttention  (5 points)
Dans le cas de la couche `DecoderAttention`, la fonction `call` prend en paramètres les entrées suivantes :
- `input` : Les entrées de la couche, correspondant à la sortie de la couche `DecoderAttention`
- `training` : Valeur booléenne indiquant si le modèle est en entraînement ou pas.

L'implémentation de la méthode est très similaire à la fonction `call` de la classe `GlobalSelfAttention`, mais diffère en un point clé : le masque causal. Ce masque permet notamment de ne pas considérer les jetons futurs lorsque le mécanisme d'attention est calculé. Cela évite au Transformer de s'entraîner en connaissant les jetons futurs qu'il doit prédire (donc en "trichant"). Cet [article](https://medium.com/analytics-vidhya/masking-in-transformers-self-attention-mechanism-bad3c9ec235c) donne plus d'information sur le masque causal.

Cette fonction doit exécuter les opérations suivantes :
1. Appliquer la couche de têtes d'attention multiples avec les bonnes valeurs de `K`, `V` et `Q` (Ne pas oublier de passer l'argument `training` à la couche et d'activer le masque causal de la couche en mettant l'attribut `use_causal_mask` à `True` lors de l'appel de la couche d'attention).
2. Ajouter la sortie de la couche de tête d'attention aux entrées à l'aide de la couche `Add`
3. Normaliser le tout à l'aide de la couche de normalisation

In [99]:
class DecoderAttention(DefaultAttention):
    """
    Couche d'attention semblable à la couche globale d'auto-attention, mais en masquant 
    les données qui viennent après
    """
    def __init__(self, **kwargs):
        """
        Initialise une couche de têtes d'attention suivie d'une couche de normalisation 
        puis d'addition
        """
        super().__init__(**kwargs)

    def call(self, input, training):
        """
        Exécute la couche d'attention en masquant les données après. Ajoute les sorties 
        d'attention à l'entrée et normalise le tout
        """
        # TODO
        Q, K, V = input, input, input
        attention_output = self.multiHeadAttention(Q, K, V, training=training, use_causal_mask=True)
        Q = self.addLayer([Q, attention_output])
        Q = self.layerNormalization(Q)
        return Q

Vous pouvez tester votre implémentation des couches d'attention à l'aide de la fonction suivante :

In [103]:
def test_attention():
    config = {
        'num_heads': 3,
        'key_dim': 3,
        'dropout': 0.1
    }
    cross_attention = CrossAttention(**config)
    global_self_attention = GlobalSelfAttention(**config)
    decoder_attention = DecoderAttention(**config)

    # Create determinisitc inputs and context
    generator = tf.random.Generator.from_seed(1)
    input = generator.normal(shape=(3, 1, 3))
    context = generator.normal(shape=(3, 1, 3))

    # Make attention layer deterministic
    layer = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=4, dropout=0.1, kernel_initializer=tf.keras.initializers.ones())
    cross_attention.multiHeadAttention = layer
    global_self_attention.multiHeadAttention = layer
    decoder_attention.multiHeadAttention = layer

    outputs_cross_attention = tf.cast(cross_attention(input, context) * 100, tf.int32)
    outputs_global_self_attention = tf.cast(global_self_attention(input) * 100, tf.int32)
    outputs_decoder_attention = tf.cast(decoder_attention(input) * 100, tf.int32)

    print('Cross Attention result : ')
    print(outputs_cross_attention, '\n')

    print('Global-Self Attention result : ')
    print(outputs_global_self_attention, '\n')

    print('Decoder Attention result : ')
    print(outputs_decoder_attention, '\n')

test_attention()

Cross Attention result : 
tf.Tensor(
[[[ 124 -119   -4]]

 [[ 140  -59  -80]]

 [[  79   60 -140]]], shape=(3, 1, 3), dtype=int32) 

Global-Self Attention result : 
tf.Tensor(
[[[ 124 -119   -4]]

 [[ 140  -59  -80]]

 [[  79   60 -140]]], shape=(3, 1, 3), dtype=int32) 

Decoder Attention result : 
tf.Tensor(
[[[ 124 -119   -4]]

 [[ 140  -59  -80]]

 [[  79   60 -140]]], shape=(3, 1, 3), dtype=int32) 



Sortie attendue : 

```
Cross Attention result : 
tf.Tensor(
[[[ 124 -119   -4]]

 [[ 140  -59  -80]]

 [[  79   60 -140]]], shape=(3, 1, 3), dtype=int32) 

Global-Self Attention result : 
tf.Tensor(
[[[ 124 -119   -4]]

 [[ 140  -59  -80]]

 [[  79   60 -140]]], shape=(3, 1, 3), dtype=int32) 

Decoder Attention result : 
tf.Tensor(
[[[ 124 -119   -4]]

 [[ 140  -59  -80]]

 [[  79   60 -140]]], shape=(3, 1, 3), dtype=int32) 
```

#### 4.3 Feed Forward  (5 points)

La couche Feed Forward est, dans notre cas, simplement une séquence de 2 couches denses, d'une couche de dropout, d'une couche d'addition et d'une couche de normalisation. Ces couches sont déjà initialisées dans le constructeur à l'aide d'une couche `Sequential` qui regroupe plusieurs couches et les applique une à la suite de l'autre.

La fonction `call` prend en paramètres les entrées suivantes :
- `input` : Entrées de la couche (varie en fonction d'où est située cette couche dans l'architecture)

Elle retourne ensuite le résultat une fois que les transformations sont appliquées sur les entrées

Elle effectue les opérations suivantes :
1. Exécute la couche séquentielle initialisée dans le constructeur
2. Ajoute le résultat de la couche séquentielle aux entrées
3. Normalise le tout à l'aide de la couche de normalisation

In [104]:
class FeedForward(tf.keras.layers.Layer):
    """
    Couche de propagation à la sortie des couches d'attention
    """

    def __init__(self, dim_model, feed_forward_size, dropout_rate=0.1):
        """
        Initialise des couches de propagation dense (avec dropout), d'addition et de normalisation
        Args :
            - dim_model : Dimension du modèle (sortie de la couche)
            - feed_forward_size : Dimension de la couche dense de propagation (entrée)
            - dropout_rate : Ratio des entrées de la couche de dropout qui 
            seront initialisés à zéro de manière aléatoire
        """
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(feed_forward_size, activation='relu'),
            tf.keras.layers.Dense(dim_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, input):
        """
        Exécute les couches de propagation sur l'entrée, additionne le tout et normalise
        """
        # TODO
        input = self.add([input, self.seq(input)])
        input = self.layer_norm(input)
        return input

#### 4.4 Encodeur (3 points)

L'encodeur de notre Transformer est composé en réalité de plusieurs couches appelées `EncoderLayer`. Ces couches représentent une seule passe d'un encodeur. Cependant, la classe `Encoder` regroupe plusieurs de ces `EncoderLayer` pour permettre au Transformer de capturer des contextes plus compliqués entre les mots.

Vous aurez donc à compléter la méthode `call` de la classe `EncoderLayer`. Cette méthode prend en entrée les paramètres suivants :
- `input` : Entrées de la couche (notamment la sortie de la classe `PositionalEmbedding`)
- `training` : Valeur booléenne indiquant si la méthode est appelée durant l'entraînement ou pas

Elle retourne les entrées une fois qu'elles sont passées à travers toutes les couches (`GlobalSelfAttention`, `FeedForward`)

Cette méthode devra exécuter les opérations suivantes :
1. Appeler la couche d'attention avec les entrées
2. Appeler la couche de propagation sur la sortie de la couche d'attention

In [113]:
class EncoderLayer(tf.keras.layers.Layer):
    """
    Classe représentant une couche d'encodeur
    """

    def __init__(self, *, dim_model, num_heads, feed_forward_size, dropout_rate=0.1):
        """
        Initialise une couche d'auto-attention suivie d'une couche de propagation

        Args :
            dim_model : Dimension des embeddings du model
            num_heads : Nombre de têtes d'attention de l'encodeur
            feed_forward_size : Nombre de neurones du feed forward
            dropout_rate : Ratio des entrées de la couche d'attention qui seront 
            initialisés à zéro de manière aléatoire
        """
        super().__init__()

        self.self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=dim_model,
            dropout=dropout_rate
        )

        self.ffn = FeedForward(dim_model, feed_forward_size)

    def call(self, input, training):
        """
        Exécute la couche d'attention et de propagation sur les entrées.
        L'argument training spécifie si l'appel est effectué durant l'entrainement 
        ou pas (important pour la couche d'attention)
        """
        # TODO
        input = self.self_attention(input, training=training)
        input = self.ffn(input)
        return input

Maintenant, la classe `Encoder` s'occupe de regrouper plusieurs `EncoderLayer` pour permettre au Transformer d'inférer des contextes plus complexes.

La méthode `call` de la classe `Encoder` prend en entrée les paramètres suivants :
- `input` : Entrées de la couche (correspondant aux indices des jetons de la phrase)
- `training` : Valeur booléenne indiquant si la méthode est appelée durant l'entraînement ou pas

Elle retourne les entrées une fois qu'elles sont passées à travers toutes les couches d'encodeur

Cette méthode exécute les opérations suivantes :
1. Appeler la couche de plongements de position sur les entrées
2. Appliquer la couche de dropout sur le résultat
3. Appeler toutes les couches `EncoderLayer` (la sortie d'une couche d'encodeur devient l'entrée d'une autre)   

In [106]:
class Encoder(tf.keras.layers.Layer):
    """
    Classe représentant tous les encodeurs du Transformer
    """

    def __init__(self, *, num_layers, dim_model, num_heads, feed_forward_size, vocab_size, dropout_rate=0.1):
        """
        Initialise la couche de plongements de position, une couche dropout et les couches d'encodeurs
        Args :
            num_layers : Nombre de couches d'encodeurs
            dim_model : Dimension des embeddings du model
            num_heads : Nombre de têtes d'attention de l'encodeur
            feed_forward_size : Dimension du feed forward (en sortie)
            vocab_size : Taille du vocabulaire (correspondant à la taille d'entrée de la 
            couche de plongements de position)
            dropout_rate : Ratio des entrées de la couche de dropout qui seront initialisés 
            à zéro de manière aléatoire
        """
        super().__init__()

        self.dim_model = dim_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(input_size=vocab_size, dim_model=dim_model)

        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.enc_layers = [EncoderLayer(dim_model=dim_model, num_heads=num_heads, feed_forward_size=feed_forward_size, dropout_rate=dropout_rate) for _ in range(num_layers)]

    def call(self, input, training):
        """
        Execute la couche de plongements et de dropout puis toutes les couches d'encodeurs
        """
        input = self.dropout(self.pos_embedding(input))
        for i in range(self.num_layers):
            input = self.enc_layers[i](input, training)

        return input


#### 4.5 Decodeur (3 points)

Le décodeur de notre Transformer est composé en réalité de plusieurs couches appelées `DecoderLayer`. Ces couches représentent une seule passe d'un décodeur. Cependant, la classe `Decoder` regroupe plusieurs de ces `DecoderLayer` pour permettre au Transformer de capturer des contextes plus compliqués entre les mots.

Vous aurez donc à compléter la méthode `call` de la classe `DecoderLayer`. Cette méthode prend en entrée les paramètres suivants :
- `input` : Entrées de la couche
- `context` : Le contexte des couches d'attention
- `training` : Valeur booléenne indiquant si la méthode est appelée durant l'entraînement ou pas
 
Elle retourne les entrées une fois qu'elles sont passées à travers toutes les couches (`DecoderAttention`, `CrossAttention`, `FeedForward`)

Cette méthode devra exécuter les opérations suivantes :
1. Appeler la couche d'attention du décodeur avec les entrées
2. Appeler la couche d'attention croisée
3. Appeler la couche de propagation (`FeedForward`)

In [120]:
class DecoderLayer(tf.keras.layers.Layer):
    """
    Classe représentant une couche de décodeur
    """

    def __init__(self, *, dim_model, num_heads, feed_forward_size, dropout_rate=0.1):
        """
        Args :
            dim_model : Dimension des embeddings du model
            num_heads : Nombre de têtes d'attention du décodeur
            feed_forward_size : Nombre de neurones du feed forward
            dropout_rate : Ratio de dropout pour les neurones de la couche de Feed Forward
        """
        super().__init__()

        self.encoder_decoder_attention = DecoderAttention(
            num_heads=num_heads,
            key_dim=dim_model,
            dropout=dropout_rate
        )

        self.cross_attention = CrossAttention(
            num_heads=num_heads,
            key_dim=dim_model,
            dropout=dropout_rate
        )

        self.ffn = FeedForward(dim_model, feed_forward_size)

    def call(self, input, context, training):
        """
        Exécute les couches d'attention suivies des couches de propagation FFN
        """
        # TODO
        input = self.encoder_decoder_attention(input, training=training)
        input = self.cross_attention(input, context, training=training)
        input = self.ffn(input)
        return input

La classe `Decoder` s'occupe de regrouper plusieurs `DecoderLayer`.

La méthode `call` de la classe `Decoder`prend en entrée les paramètres suivants :
- `input` : Entrées de la couche (correspondant aux indices des jetons de la phrase)
- `context` : Contexte des couches d'attention (correspondant à la sortie de l'encodeur)
- `training` : Valeur booléenne indiquant si la méthode est appelée durant l'entraînement ou pas

Elle retourne les entrées une fois qu'elles sont passées à travers toutes les couches de décodeur

Cette méthode  exécute les opérations suivantes :
1. Appeler la couche de plongements de position sur les entrées
2. Appliquer la couche de dropout sur le résultat
3. Appeler les couches `DecoderLayer` successivement


In [115]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, dim_model, num_heads, feed_forward_size, vocab_size, dropout_rate=0.1):
        """
        Initialise la couche de plongements de position, une couche dropout et les couches d'encodeurs
        Args :
            num_layers : Nombre de couches de décodeur
            dim_model : Dimension des embeddings du model
            num_heads : Nombre de têtes d'attention de l'encodeur
            feed_forward_size : Dimension du feed forward (en sortie)
            vocab_size : Taille du vocabulaire (correspondant à la taille d'entrée 
            de la couche de plongements de position)
            dropout_rate : Ratio des entrées de la couche de dropout qui seront 
            initialisés à zéro de manière aléatoire
        """
        super().__init__()

        self.dim_model = dim_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(input_size=vocab_size, dim_model=dim_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_layers = [DecoderLayer(dim_model=dim_model, num_heads=num_heads, feed_forward_size=feed_forward_size, dropout_rate=dropout_rate) for x in range(self.num_layers)]

        self.last_attn_scores = None

    def call(self, input, context, training):
        """
        Execute la couche de plongements et de dropout 
        puis toutes les couches de décodeurs
        """
        input = self.dropout(self.pos_embedding(input))
        for i in range(self.num_layers):
            input = self.dec_layers[i](input, context, training)

        return input

#### 4.6 Transformer (4 points)

Le Transformer est maintenant prêt à être créé. Le constructeur s'occupe déjà d'initialiser tous les attributs nécessaires à son fonctionnement.

La fonction `call` prend en entrées les arguments suivants :
- `inputs` : Les entrées du modèle de la forme d'un tuple regroupant l'entrée sparql et l'entrée anglaise (`inputs = (sparql, english)`)
- `training` : Valeur booléenne indiquant si le modèle est en entrainement ou pas

La méthode `call` doit :
1. Séparer les entrées reçues en sparql et english
2. Envoyer les phrases sparql à l'encodeur
3. Envoyer les phrases en anglais au décodeur avec comme contexte la sortie de l'encodeur
4. Envoyer la sortie du décodeur à la couche dense initialisée dans le constructeur (`self.dense_layer`)
5. Appeler la fonction `drop_mask` avec comme argument les probabilités générées par la couche dense (en enlevant l'attribut `_keras_mask` des probabilités générées par la couche dense, on évite au modèle d'utiliser ce masque lorsqu'il calcule les métriques et le coût)

In [136]:
class Transformer(tf.keras.Model):
    """
    Classe représentant le Transformer
    """

    def __init__(self, *, num_layers, dim_model, num_heads, feed_forward_size,
                input_vocab_size, target_vocab_size, dropout_rate=0.1):
        """
        Initialise les couches d'encodeur et de décodeurs et la couche dense finale
        Args :
            num_layers : Nombre de couches de décodeur
            dim_model : Dimension des embeddings du model
            num_heads : Nombre de têtes d'attention de l'encodeur et du décodeur
            feed_forward_size : Dimension du feed forward (en sortie)
            input_vocab_size : Taille du vocabulaire d'entrée
            target_vocab_Size : Taille du vocabulaire de sortie
            dropout_rate : Ratio des entrées de la couche de dropout qui seront 
            initialisés à zéro de manière aléatoire
        """
        super().__init__()
        self.encoder = Encoder(num_layers=num_layers, dim_model=dim_model,
                            num_heads=num_heads, feed_forward_size=feed_forward_size,
                            vocab_size=input_vocab_size,
                            dropout_rate=dropout_rate)

        self.decoder = Decoder(num_layers=num_layers, dim_model=dim_model,
                            num_heads=num_heads, feed_forward_size=feed_forward_size,
                            vocab_size=target_vocab_size,
                            dropout_rate=dropout_rate)

        self.dense_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, training=True):
        """
        Appel les couches d'encodeur et de décodeur avec les bonnes entrées et 
        contexte ainsi que la couche dense finale
        """
        # TODO
        sparql, english = inputs
        sparql = self.encoder(sparql, training=training)
        english = self.decoder(english, sparql, training=training)
        english = self.dense_layer(english)
        self.drop_mask(training, english)
        return english

    def drop_mask(self, training, probabilities):
        if not training:
            try:
                del probabilities._keras_mask
            except AttributeError:
                pass

Vous pouvez tester votre implémentation finale du Transformer avec la fonction suivante. **Attention, ce n'est pas parce que vous obtenez les bons résultats qu'il n'y a pas de bugs dans votre implémentation, mais c'est déjà un bon signe**

In [137]:
def test_transformer():

    config = {
        'num_layers': 2,
        'dim_model': 2,
        'num_heads': 2,
        'feed_forward_size': 2,
        'input_vocab_size': 2,
        'target_vocab_size': 2,
        'dropout_rate': 0.1
    }

    initializer = tf.keras.initializers.glorot_normal(42)

    feed_forward = FeedForward(2, 2, 0.1)
    feed_forward.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(2, activation='relu', kernel_initializer=initializer, use_bias=False),
        tf.keras.layers.Dense(2, kernel_initializer=initializer, use_bias=False),
        tf.keras.layers.Dropout(0.1, seed=42)
    ])
    feed_forward.add = tf.keras.layers.Add()
    feed_forward.layer_norm = tf.keras.layers.LayerNormalization(beta_initializer=initializer, gamma_initializer=initializer)

    transformer = Transformer(**config)

    transformer.encoder.pos_embedding.embedding_layer = tf.keras.layers.Embedding(2, 2, embeddings_initializer=initializer, mask_zero=False)
    for l in transformer.encoder.enc_layers:
        l.self_attention = GlobalSelfAttention(num_heads=2, key_dim=2, dropout=0.1, kernel_initializer=initializer)
        l.ffn = feed_forward
    transformer.encoder.dropout = tf.keras.layers.Dropout(0.1, seed=42)

    transformer.decoder.pos_embedding.embedding_layer = tf.keras.layers.Embedding(2, 2, embeddings_initializer=initializer, mask_zero=True)
    for l in transformer.decoder.dec_layers:
        l.cross_attention = CrossAttention(num_heads=2, key_dim=2, dropout=0.1, kernel_initializer=initializer)
        l.encoder_decoder_attention = DecoderAttention(num_heads=2, key_dim=2, dropout=0.1, kernel_initializer=initializer)
        l.ffn = feed_forward
        
    transformer.dense_layer = tf.keras.layers.Dense(2, kernel_initializer=initializer, use_bias=False)
    transformer.decoder.dropout = tf.keras.layers.Dropout(0.1, seed=42)

    # Create determinisitc inputs and context
    generator = tf.random.Generator.from_seed(1)
    input = generator.normal(shape=(2, 2, 2))
    context = generator.normal(shape=(2, 2, 2))

    input_transformer = (input, context)
    output = transformer(input_transformer, training=True)
    print(output)

test_transformer()

tf.Tensor(
[[[[ 0.00026987 -0.03291528]
   [ 0.00026987 -0.03291528]]

  [[ 0.00026987 -0.03291528]
   [ 0.00026987 -0.03291528]]]


 [[[ 0.00026988 -0.03291521]
   [ 0.00026984 -0.03291551]]

  [[ 0.00026987 -0.03291528]
   [ 0.00026988 -0.03291521]]]], shape=(2, 2, 2, 2), dtype=float32)


```
tf.Tensor(
[[[[ 0.00026983 -0.03291529]
   [ 0.00026987 -0.03291498]]

  [[ 0.00026987 -0.03291498]
   [ 0.00026987 -0.03291498]]]


 [[[ 0.00026983 -0.03291529]
   [ 0.00026987 -0.03291498]]

  [[ 0.00026987 -0.03291498]
   [ 0.00026987 -0.03291498]]]], shape=(2, 2, 2, 2), dtype=float32)
```

#### 4.7 Scheduler

La classe `Scheduler` permet entre autre de mettre à jour le taux d'apprentissage du modèle lors de l'entraînement. Son implémentation complète vous est fournie.

In [138]:
class Scheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, dim_model, warmup_steps):
        super().__init__()
        self.dim_model = tf.cast(dim_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        return tf.math.rsqrt(self.dim_model) * tf.math.minimum(tf.math.rsqrt(step), step * (self.warmup_steps ** -1.5))
    
    def get_config(self):
        config = {
            'd_model': self.dim_model,
            'warmup_steps': self.warmup_steps,
        }
        return config

### 5. Entrainement du modèle (15 points)

Il est maintenant temps de créer le traducteur qui va transformer des requêtes sparql en anglais. Pour cela, il faudra compléter 4 méthodes de la classe `Translator`, soit les méthodes `prepare`, `fit` et `translate`.
___

La fonction `prepare` reçoit les données d'entrainement et de validation sous forme de pandas DataFrame et s'occupe de :
1. Appeler le préprocesseur sur les données d'entrainement et de validation
2. Créer un objet `tf.Dataset` contenant un tuple des requêtes sparql et des questions en anglais pour l'ensemble d'entraînement et de validation
3. Envoyer les 2 datasets créés (entraînement et validation) au batcher

Elle retourne un tuple contenant les batches d'entraînement et de validation

___

La fonction `fit` s'occupe simplement d'entraîner le modèle avec les données d'entraînement et de validation passés en paramètre.
___

La fonction `translate` s'occupe de traduire une série de données sparql en anglais. Pour cela, plusieurs étapes doivent être effectuées. Elle doit :
1. Appliquer le préprocesseur sur l'ensemble de test donné
2. Créer des batches à l'aide du batcher de test
3. Pour chaque valeur dans les batches créés
  - Extraire le contenu du tuple. Souvenez-vous que ce qui est ressorti par la méthode `prepare_batch` dans le cas d'un batcher de test est une tuple de la forme (phrase SPARQL, phrase anglais) où initialement, la phrase anglaise est initialisée avec le jeton de départ
  - Envoyer les contextes et les phrases au Transformer pour qu'il prédise le prochain jeton
  - Concaténer ensemble tous les jetons prédits par le Transformer pour générer la traduction  
4. Réduire la taille des prédictions pour enlever tout ce qui vient après le jeton de fin généré par le Transformer (si aucun jeton de fin n'est généré, la traduction n'a pas besoin d'être coupée)
5. Transformer les jetons prédits en mots à l'aide du bon tokenizer
6. Annuler les transformations initiales effectuées à l'aide du pré-traitement


Les fonctions `masked_loss` et `masked_accuracy` vous sont fournies et permettent d'évaluer l'exactitude du Transformer en évaluant une fonction de perte propre au Transformer. 

In [ ]:
class Translator:

    num_layers = 4
    dim_model = 128
    feed_forward_size = 512
    num_heads = 6
    dropout_rate = 0.1
    input_vocab_size = 8000
    target_vocab_size = 8000
    batch_size = 64
    batch_size_test = 500
    buffer_size = 20000
    buffer_size_test = None

    def __init__(self):
        """
        Initialise le preprocessor, les tokenizers, les batchers et le Transformer 
        avec les bons paramètres
        """

        self.pre_processor = Preprocessor()

        self.tokenizers = GroupedTokenizers(
            LanguageTokenizer.reserved_tokens,
            root + 'language_vocab_english.txt',
            root + 'language_vocab_sparql.txt'
        )

        self.train_batcher = Batcher(tokenizers=self.tokenizers, train=True, max_tokens=Translator.dim_model, batch_size=Translator.batch_size, buffer_size=Translator.buffer_size)
        self.test_batcher = Batcher(tokenizers=self.tokenizers, train=False, max_tokens=Translator.dim_model, batch_size=Translator.batch_size_test, buffer_size=Translator.buffer_size_test)

        self.transformer = Transformer(
            num_layers=Translator.num_layers,
            dim_model=Translator.dim_model,
            num_heads=Translator.num_heads,
            feed_forward_size=Translator.feed_forward_size,
            input_vocab_size=Translator.input_vocab_size,
            target_vocab_size=Translator.target_vocab_size,
            dropout_rate=Translator.dropout_rate)

        self.scheduler = Scheduler(Translator.dim_model, 4000)
        self.optimizer = tf.keras.optimizers.Adam(self.scheduler, beta_1=0.9, beta_2=0.95, epsilon=1e-9)

        self.transformer.compile(
            loss=Translator.masked_loss,
            optimizer=self.optimizer,
            metrics=[Translator.masked_accuracy])

        self.end = self.tokenizers.sparql.tokenize([''])[0][1][tf.newaxis]

    def prepare(self, train: pd.DataFrame, val: pd.DataFrame):
        """
        Prépare les ensembles de validation et d'entrainement à l'entrainement 
        en les envoyant au preprocessor et au batcher
        Args :
            - train : DataFrame d'entrainement avec les columns sparql (entrée) et anglais (sortie)
            - val : DataFrame de validation avec les columns sparql (entrée) et anglais (sortie)

        Returns :
            Tuple contenant les batches d'entraînement et les batches de validation
        """
        # TODO
        pass

    def fit(self, training, validation, epochs=50):
        """
        Entraine le modèle en utilisant l'ensemble d'entrainement et valide le résultat
        """
        # TODO
        pass

    def translate(self, sparql: pd.Series):
        """
        Traduit une série de requêtes sparql en anglais
        """
        # TODO
        pass

    def masked_loss(label, pred):
        mask = label != 0
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')
        loss = loss_object(label, pred)

        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask

        loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
        return loss

    def masked_accuracy(label, pred):
        pred = tf.argmax(pred, axis=2)
        label = tf.cast(label, pred.dtype)
        match = label == pred

        mask = label != 0

        match = match & mask

        match = tf.cast(match, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(match)/tf.reduce_sum(mask)

#### 5.1 Préparation des données 

Exécuter ensuite la cellule ci-dessous pour créer maintenant une instance de la classe `Translator`, charger les données d'entraînement et de validation et préparer les données à l'entraînement

In [ ]:
translator = Translator()

data_loader = DataLoader(
    training_path=root + 'train.csv',
    validation_path=root + 'validation.csv'
)

train_batch, val_batch = translator.prepare(data_loader.train, data_loader.val)

#### 5.2 Entraînement 

Entraînez le modèle avec les données

In [ ]:
translator.fit(train_batch, val_batch)

#### 5.3 Traduction 

Effectuez la traduction des données de test pour valider l'efficacité du modèle

In [ ]:
predictions = translator.translate(data_loader.val['sparql'])
formatted_predictions = pd.concat([pd.DataFrame(predictions), data_loader.val], axis=1)
formatted_predictions.drop(['id', 'sparql'], inplace=True, axis=1)
formatted_predictions.rename(columns={0:'prediction', 'english':'target_text'}, inplace=True)
formatted_predictions.head(10)

### 6. Évalution : Métrique BLEU (15 points)

Pour évaluer l'efficacité des traductions, la métrique BLEU sera utilisée. La formule est donnée ci-dessous : 
$$BLEU = BP * exp \Big( \sum_{n=1}^{N} w_n log p_n \Big)$$

où $p_n$ est la précision modifiée pour le n-gramme (correspondant au ratio de la fréquence maximum du n-gramme dans chaque phrase de référence par la fréquence du n-gramme). Posons ensuite $r$ comme le nombre de mots dans la phrase cible et $c$ comme le nombre de mots dans la phrase prédite. Si $c>r$, alors BP vaut 1. Sinon $BP = exp(1 - \frac{r}{c})$.

Les valeurs des poids $w_n$ est ce qui donne les différentes variations de la métrique BLEU. Dans notre cas, la métrique BLEU-3 sera utilisée. 

In [ ]:
def evaluate_model(data: pd.DataFrame):
    """
    Évalue la précision du modèle en utilisant la métrique BLEU
    Args :
        - data : DataFrame contenant deux colonnes (predictions et target_text)

    Returns :
        La moyenne du score BLEU
    """
    weights = (1/3, 1/3, 1/3) # Use Bleu-3
    scores = np.zeros(data.shape[0])
    index = 0
    for iter, row in data.iterrows():
        if not pd.notnull(row['prediction']):
            continue
        prediction = row['prediction'].split()
        target_text = row['target_text'].split()

        scores[index] = sentence_bleu([target_text], prediction, weights=weights)

        index += 1
    return np.mean(scores)


#### 6.1 Évaluation du modèle 

Appelez la fonction `evaluate_model` sur les prédictions de votre modèle pour évaluer sa performance.

In [ ]:
evaluate_model(formatted_predictions)

#### 6.2. Analyse des erreurs (10 points)
Analysez les traductions du modèle et ses erreurs. Implantez une analyse statistique  (selon la forme de votre choix) qui affiche des catégories d'erreurs et leur % d'occurrence parmi l'ensemble des erreurs possibles. Vous pouvez orienter votre fonction pour qu'elle décrive des dimensions spécifiques. Par exemple : les erreurs sont-elles plus souvent sur les éléments de la base de connaissance "dbx" ou sur le reste des jetons ? Les erreurs sont-elles dues à des éléments qui ne sont pas vus en entrainement ?


> TODO

#### 6.3 Amélioration (5 points)
Donnez des pistes de solution pour améliorer le score BLEU

> TODO

## LIVRABLES:
Vous devez remettre sur Moodle un zip contenant les fichiers suivants :

1-	Le code : Vous devez compléter le squelette inf8460_tp3.ipynb sous le nom   equipe_i_inf8460_TP3.ipynb (i = votre numéro d’équipe). Indiquez vos noms et matricules au début du notebook. Ce notebook doit contenir les fonctionnalités requises avec des commentaires appropriés. Le code doit être exécutable sans erreur et accompagné de commentaires appropriés de manière à expliquer les différentes fonctions. Les critères de qualité tels que la lisibilité du code et des commentaires sont importants. Tout votre code et vos résultats doivent être exécutables et reproductibles ; 

2-	Un fichier pdf représentant votre notebook complètement exécuté sous format pdf. 
Pour créer le fichier cliquez sur File > Download as > PDF via LaTeX (.pdf). Assurez-vous que le PDF est entièrement lisible.


## EVALUATION 
Votre TP sera évalué selon les critères suivants :

1. Exécution correcte du code
2. Qualité du code (noms significatifs, structure, performance, gestion d’exception, etc.)
3. Commentaires clairs et informatifs
4. Performance attendue des modèles
5. Réponses correctes/sensées aux questions de réflexion ou d'analyse
 
